In [ ]:
import pandas as pd

df = pd.read_csv('/home/ryanrodr/Downloads/codigos_distritos_msp.csv', encoding='latin1', sep=';')
df.head(3)

In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd

# Lista dos distritos de São Paulo (exemplo)
distritos = df['distrito']

# Inicialize o geolocalizador
geolocator = Nominatim(user_agent="geoapiExercises")

# Função para obter latitude e longitude
def obter_lat_long(distrito):
    try:
        location = geolocator.geocode(distrito + ", São Paulo, Brasil")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erro ao obter coordenadas para {distrito}: {e}")
        return None, None

# Crie uma lista para armazenar os resultados
resultados = []

# Obtenha coordenadas para cada distrito
for distrito in distritos:
    lat, long = obter_lat_long(distrito)
    resultados.append({
        "Distrito": distrito,
        "Latitude": lat,
        "Longitude": long
    })

# Crie um DataFrame para visualizar os resultados
df2 = pd.DataFrame(resultados)
df2 = df2.dropna()

In [ ]:
from meteostat import Point, Daily
from datetime import datetime

# Definir datas diretamente no código
start = datetime(1950, 1, 1)
end = datetime(2024, 8, 31)

# Lista para armazenar os dados de cada distrito
all_data = []

# Iterar sobre as linhas do DataFrame e obter dados do Meteostat
for index, row in df2.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    location = Point(latitude, longitude)
    
    # Obter dados diários do Meteostat
    data = Daily(location, start, end).fetch()
    
    # Adicionar uma coluna com o nome do distrito
    data['Distrito'] = row['Distrito']
    
    # Adicionar os dados à lista
    all_data.append(data)

# Concatenar todos os DataFrames em um único DataFrame
final_data = pd.concat(all_data)

# Resetar o índice do DataFrame final
final_data.reset_index(inplace=True)

final_data